# Clustering of VGG features

In [ ]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 
import time
# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
from skimage import io

from find_maxima import find_maxima
import tifffile
import glob

from PIL import Image
import sys
sys.path.append('../src')
from convert_ROI_to_cell_volumes import *

from scipy import ndimage
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from sklearn.manifold import TSNE
from PIL import Image
         
            
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

### TSNE and Kmeans prediction of images.

### Trains from image volumes (without time)

In [ ]:
outpaths = ['/Users/dominicwaithe/Desktop/special/']

#pca = pickle.load(open( "/Users/dominicwaithe/Desktop/cell_images/pca.pickle", "rb" ))
#kmeans = pickle.load(open( "/Users/dominicwaithe/Desktop/cell_images/kmeans.pickle", "rb" ))
channel = 1
cell_data = []

imgnum = 0
xs = []
store_cell_feats = []
for path in evalpaths:
    data,roi_array = collect_info(path,channel,'raw')
    
    for cell in range(0,data.__len__()):
        
        img_vol = data[cell]
        cell_shape = img_vol.__len__()
        raw_img = img_vol[cell_shape//2]
        nimg = normalise_for_8bit(raw_img)
        img = Image.fromarray(np.uint8(nimg))
        img = img.resize((224, 224), Image.ANTIALIAS)
        
        img = np.array(img)
        reshaped_img = np.zeros((1,224,224,3))
        reshaped_img[0,:,:,0] = img
        reshaped_img[0,:,:,1] = img
        reshaped_img[0,:,:,2] = img
        # prepare image for model
        imgx = preprocess_input(reshaped_img)
        # get the feature vector
        features = model.predict(imgx, use_multiprocessing=True)
        store_cell_feats.append(features)
        

feat = np.array(list(store_cell_feats))
feat = feat.reshape(-1,4096)
# reduce the amount of dimensions in the feature vector
pca = PCA(n_components=10, random_state=22)
pca.fit(feat)


#_TSNE = TSNE(n_components=2)
#x = _TSNE.fit_transform(feat)

#x = pca.transform(feat)

# cluster feature vectors
kmeans = KMeans(n_clusters=4,n_jobs=-1, random_state=22)
kmeans.fit(x)
x = pca.transform(feat)
xs.append(x)
cell_data.append(kmeans.predict(x))
copy_to_clipboard(cell_data)

In [ ]:

def copy_to_clipboard(cell_data):
    stg = ""
    maxt = 0
    for num in range(0,cell_data.__len__()):
        if cell_data[num].__len__() > maxt:
            maxt = cell_data[num].__len__()
    for idx in range(0,maxt):
        for num in range(0,cell_data.__len__()):
            if idx < cell_data[num].__len__():
                stg += str(cell_data[num][idx]) 
            stg+="\t"
        stg+='\n'
    pyperclip.copy(stg)
    spam = pyperclip.paste()

In [ ]:
x = xs[0]
labels = cell_data[0]
bint = labels==0
plt.plot(x[bint,0],x[bint,1],'x')
bint = labels==1
plt.plot(x[bint,0],x[bint,1],'bo')
bint = labels==2
plt.plot(x[bint,0],x[bint,1],'ko')
bint = labels==3
plt.plot(x[bint,0],x[bint,1],'+')

### Timeseries data
Train from timeseries. Pools all cellular images from dataset for clustering.

In [ ]:
#timeseries
#Train from timeseries. Pools all cellular images from dataset for clustering.

train_paths = ['/Users/dominicwaithe/Desktop/focused_stkoutMD2join_0016/']
channel = 0
cell_data = []

imgnum = 0
xs = []
cell_img = []
train_array_feats = []


for path in train_paths:
    data, roi_array = collect_info(path,channel,'raw')
    
    for cell in range(0,data.__len__()):
        
        img_vol = data[cell]
        cell_shape = img_vol.__len__()
       
        if cell_shape > 200:
            reshaped_img = np.zeros((cell_shape,224,224,3))
            for i, raw_img in enumerate(img_vol):
                t1 = time.time()
                nimg = normalise_for_8bit(raw_img)
                img = Image.fromarray(np.uint8(nimg))
                img = img.resize((224, 224), Image.NEAREST,reducing_gap=3)
                img = np.array(img)
                
                
                reshaped_img[i,:,:,0] = img
                reshaped_img[i,:,:,1] = img
                reshaped_img[i,:,:,2] = img
                cell_img.append(img)
                # prepare image for model
            imgx = preprocess_input(reshaped_img)
            t2 = time.time()
            # get the feature vector
            features = model.predict(imgx, use_multiprocessing=True)
            train_array_feats.extend(features)
            
            t3 = time.time()
            print('time',t2-t1,t3-t2)

feat = np.array(list(train_array_feats))
feat = feat.reshape(-1,4096)
pca = PCA(n_components=10, random_state=22)
pca.fit(feat)
x = pca.transform(feat)
xs.append(x)
kmeans = KMeans(n_clusters=3,n_jobs=-1, random_state=22)
kmeans.fit(x)

pickle.dump(pca,open( "pca_timeseries.pickle", "wb" ))
pickle.dump(kmeans,open( "kmeans_timeseries.pickle", "wb" ))

cell_data.append(kmeans.predict(x))

### Timeseries data
Evaluates timeseries.

In [ ]:
#timeseries
#Evaluate with 
eval_paths = ['/Users/dominicwaithe/Desktop/focused_stkoutMD2join_0012/']

pca = pickle.load(open( "pca_timeseries.pickle", "rb" ))
kmeans = pickle.load(open( "kmeans_timeseries.pickle", "rb" ))
channel = 0
cell_data = []

imgnum = 0
xs = []
cell_img = []
#flowers_sub.sort()
for path in eval_paths:
    data, roi_array = collect_info(path,channel,'raw')
    
    for cell in range(0,data.__len__()):
        
        store_cell_feats = []
        img_vol = data[cell]
        cell_shape = img_vol.__len__()
        
        if cell_shape > 200:
            reshaped_img = np.zeros((cell_shape,224,224,3))
            for i, raw_img in enumerate(img_vol):
                t1 = time.time()
                nimg = normalise_for_8bit(raw_img)
                img = Image.fromarray(np.uint8(nimg))
                img = img.resize((224, 224), Image.NEAREST,reducing_gap=3)
                img = np.array(img)
                
                
                reshaped_img[i,:,:,0] = img
                reshaped_img[i,:,:,1] = img
                reshaped_img[i,:,:,2] = img
                cell_img.append(img)
                # prepare image for model
            imgx = preprocess_input(reshaped_img)
            t2 = time.time()
            # get the feature vector
            features = model.predict(imgx, use_multiprocessing=True)
            store_cell_feats.extend(features)
            
            t3 = time.time()
            print('time',t2-t1,t3-t2)
        if store_cell_feats.__len__()>0:
            feat = np.array(list(store_cell_feats))
            feat = feat.reshape(-1,4096)
            x = pca.transform(feat)
            xs.append(x)
            cell_data.append(kmeans.predict(x))


### Visualises clustering.
Train from timeseries. Pools all cellular images from dataset for clustering.

In [ ]:
#from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
for dc in range(0,xs.__len__()):
    i =0
    dim0 = 0
    dim1 = 1
    #dc = 2
   
    
    if xs[dc].__len__() >15:
        #fig = plt.figure()
        #plt.ylim(-30,30)
        #plt.xlim(-60,60)
        #ax  = Axes3D(fig)


        for xy,clu in zip(xs[dc],cell_data[dc]):
            b = [xy[dim0],xy[dim1]]
           
            if clu == 0:
                
                #ax.scatter(b[0], b[1], i,marker='o',color='c')
                #.plot(b[0],i,'cx')
                plt.plot(dc,i,'yx')
                #plt.plot(b[0],b[1],'co',alpha=0.2)
                pass
            if clu == 1:
                #ax.scatter(b[0], b[1], i,marker='o',color='b')
                #plt.plot(b[0],i,'bo')
                plt.plot(dc,i,'bo')
                #plt.plot(b[0],b[1],'bo',alpha=0.2)
                pass
            if clu == 2:
                #ax.scatter(b[0], b[1], i,marker='o',color='k')
                #plt.plot(b[0],i,'ko')
                plt.plot(dc,i,'ko')
                #plt.plot(b[0],b[1],'ko',alpha=0.2)
                pass
            if clu == 3:
                
                #ax.scatter(b[0], b[1], i,marker='o',color='y')
                #plt.plot(b[0],i,'yo')
                plt.plot(dc,i,'yo')
                
                #plt.plot(b[0],b[1],'yo',alpha=0.2)
                pass
            #if i >0:
                #rgb = np.array([np.random.random(),np.random.random(),np.random.random()])#
                #rgb = (1-(float(i)/float(256)))
                #ax.plot3D([c[0],b[0]], [c[1],b[1]], [i-1,i], color=rgb )# Data for three-dimensional scattered points

                
                #if np.sqrt((b[0]-c[0])**2+ (b[1]-c[1])**2) <20:
                #plt.arrow(c[0],c[1],b[0]-c[0],b[1]-c[1],color=[rgb,rgb,rgb],width=0.1,alpha=1)
            c = b
            i +=1/6


In [ ]:
# function that lets you view a cluster (based on identifier)        
def view_cluster(cell_data,cell_img,cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    indices  = np.where(np.array(cell_data) == cluster)[0]
    np.random.shuffle(indices)
    
    files = np.array(cell_img)[indices]
    # only allow up to 30 images to be shown at a time
    if len(files) > 100:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:99]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        
        img = np.array(file)
        plt.imshow(img)
        plt.axis('off')


In [ ]:
view_cluster(cell_data[0],cell_img,0)



In [ ]:
view_cluster(cell_data[0],cell_img,1)


In [ ]:
view_cluster(cell_data[0],cell_img,2)
